In [1]:
import pandas as pd

In [115]:
df = pd.read_csv("data/incidents_train.csv", index_col=False, dtype=str)

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5082 entries, 0 to 5081
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        5082 non-null   object
 1   year              5082 non-null   object
 2   month             5082 non-null   object
 3   day               5082 non-null   object
 4   country           5082 non-null   object
 5   title             5082 non-null   object
 6   text              5082 non-null   object
 7   hazard-category   5082 non-null   object
 8   product-category  5082 non-null   object
 9   hazard            5082 non-null   object
 10  product           5082 non-null   object
dtypes: object(11)
memory usage: 436.9+ KB


In [118]:
df = df.rename(columns={"Unnamed: 0": "id", "hazard-category": "hazard_category", "product-category": "product_category"})

In [119]:
df.tail(10)

,id,year,month,day,country,title,text,hazard_category,product_category,hazard,product
5072,5973,2022,7,23,ca,Certain That's it. brand Dark Chocolate Truffl...,Food recall warning Certain That's it. brand D...,allergens,"cocoa and cocoa preparations, coffee and tea",milk and products thereof,dark chocolates
5073,5974,2022,7,25,ie,Undeclared Cashew Nuts in Several Batches of B...,Undeclared Cashew Nuts in Several Batches of B...,allergens,"soups, broths, sauces and condiments",cashew,prepared dish
5074,5975,2022,7,25,ca,Certain Organic Traditions brand dark chocolat...,Notification Certain Organic Traditions brand ...,allergens,confectionery,milk and products thereof,confectionery
5075,5976,2022,7,26,us,Wismettac Asian Foods Issues Allergy Alert on ...,"Wismettac Asian Foods, Inc., Santa Fe Springs,...",allergens,"soups, broths, sauces and condiments",soybeans and products thereof,soups
5076,5977,2022,7,27,ca,Vicentina Fine Foods Gourmet brand Meat Lasagn...,Food recall warning Vicentina Fine Foods Gourm...,allergens,prepared dishes and snacks,cashew,lasagne
5077,5979,2022,7,28,hk,Imported biscuit may contain allergen (peanuts),Imported biscuit may contain allergen (peanuts...,allergens,cereals and bakery products,peanuts and products thereof,biscuits
5078,5980,2022,7,28,us,Danny’s Sub and Pizza Recalls Meat Pizza Produ...,023-2022\n\n \n High - Class I\n\n Produc...,fraud,prepared dishes and snacks,inspection issues,pizza
5079,5981,2022,7,29,us,Lyons Magnus Voluntarily Recalls 53 Nutritiona...,"FRESNO, Calif. – July 28, 2022 – Lyons Magnus ...",biological,non-alcoholic beverages,cronobacter spp,non-alcoholic beverages
5080,5982,2022,7,30,us,"Conagra Brands, Inc., Recalls Frozen Beef Prod...",025-2022\n\n \n High - Class I\n\n Misbra...,allergens,"meat, egg and dairy products",eggs and products thereof,frozen beef products
5081,5983,2022,7,30,us,Rachael’s Food Corporation Recalls Ready-To-Ea...,024-2022\n\n \n High - Class I\n\n Produc...,biological,prepared dishes and snacks,listeria monocytogenes,wraps


In [7]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-08-28 13:27:33--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.3’

minsearch.py.3      100%[===================>]   3.74K  --.-KB/s    in 0.002s  

2024-08-28 13:27:34 (2.05 MB/s) - ‘minsearch.py.3’ saved [3832/3832]



In [8]:
import minsearch

In [128]:
documents = df.to_dict(orient="records")

In [133]:
index = minsearch.Index(
    text_fields=['country', 'title', 'text', 'hazard_category', 'product_category', 'hazard', 'product'],
    keyword_fields=['id']
)

In [134]:
index.fit(documents)

In [135]:
def search(query):
    results = index.search(
        query=query,
        num_results=5
    )
    return results

In [136]:
search("main allergens")

[{'id': '5966',
  'year': '2022',
  'month': '7',
  'day': '19',
  'country': 'us',
  'title': 'Albertsons Companies Voluntarily Recalls Select ReadyMeals Seafood Products Due to Undeclared Allergens',
  'text': 'BOISE, Idaho – July 19, 2022 – Albertsons Companies (NYSE: ACI) has voluntarily recalled three ReadyMeals seafood items prepared in store due to allergens not listed on the ingredient labels. The recall was initiated after an internal technical review identified missing ingredients containing allergens. Consumers who have certain food allergies or severe sensitivities run the risk of serious or life - threatening allergic reactions if they consume these products and are urged not to consume these items. These consumers should discard the products or return them to their local store for a full refund. There have been no reports of injuries or adverse reactions due to consumption of these products. Anyone concerned about an injury or illness should contact a healthcare provider.

In [14]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [15]:
documents[0]

{'year': '1994',
 'month': '1',
 'day': '7',
 'country': 'us',
 'title': 'Recall Notification: FSIS-024-94',
 'text': "Case Number: 024-94   \n            Date Opened: 07/01/1994   \n            Date Closed: 09/22/1994 \n    \n            Recall Class:  1   \n            Press Release (Y/N):  Y  \n    \n            Domestic Est. Number:  05893  P   \n              Name:  GERHARD'S NAPA VALLEY SAUSAGE\n    \n            Imported Product (Y/N):  N       \n            Foreign Estab. Number:  N/A\n    \n            City:  NAPA    \n            State:  CA   \n            Country:  USA\n    \n            Product:  SMOKED CHICKEN SAUSAGE\n    \n            Problem:  BACTERIA   \n            Description: LISTERIA\n    \n            Total Pounds Recalled:  2,894   \n            Pounds Recovered:  2,894",
 'hazard_category': 'biological',
 'product_category': 'meat, egg and dairy products',
 'hazard': 'listeria monocytogenes',
 'product': 'smoked sausage'}

In [22]:
entry_template = """
 'title': {title}
 'hazard_category': {hazard_category}
 'product_category': {product_category}
 'hazard': {hazard}
 'product': {product}
""".strip()

def build_prompt(query, search_results):
    prompt_template = """
You're a food hazard detection assistant. Answer the QUESTION based on the CONTEXT from the food-incident reports.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [17]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [28]:
#query = "Which allergen contain imported biscuits"
query="What are the products that contain listeria monocytogenes?"
#query="What is the main hazard found in pizza?"

In [29]:
print(rag(query))

Based on the context from food-incident reports, the products that contain listeria monocytogenes are sausage products (Mississippi Firm Recall example), chicken based products such as Konspol Chicken Burgers and various cooked diced chicken meat products. Additionally, Glacial Treasure brand's corned beef products from the third context were also recalled for listeria monocytogenes contamination within meat product category including precooked cooked beef meats as well.


In [34]:
print(rag("Which allergen do biscuits contain?"))

Based on the food-incident reports provided, almonds are identified as an undeclared allergen in a prepackaged brand of biscuits from Woolworths Limited. Therefore, people with allergies to almonds should avoid these biscuits due to this undeclared allergen hazard.


In [33]:
print(rag("tell me the main biological hazard found in smoked sausage"))

Based on the CONTEXT provided from food-incident reports concerning smoked sausages: The main biological hazard found in smoked sausage is listeria monocytogenes. This information comes from Recall Notification titles FSIS-003-95 and FSIS-024-94, which mention the specific product (smoked sausage) and associated hazardous biological agent being identified as causing concern in these cases.


In [114]:
print(rag("What types of foreign bodies were found in products?"))

The types of food hazards involving foreign bodies found in the discussed incidents included presence of ceramic pieces as well as unspecified harmful extraneous material. The affected items were a variety including breaded chicken products, frozen poultry products specifically pilgrim's pride corp.'s products and gnp company's range which includes primarily baked goods with further distinction in the form of dough-based or readymade pie shell products used as fillers. Besides these incidents regarding meat products like chicken items from Tyson Foods Inc., pilgrim&rsquo;s pride corp., and GNP Company, a milk product recall was also announced due to potential foreign body contamination by Agropur Cooperative.
